<a href="https://colab.research.google.com/github/michael-wettach/pythonsamples/blob/main/Python_6_Git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Module zur Interaktion mit git Repositories</h1>

Eine allgemeine Dokumentation zu Git gibt es kostenlos als elektronisches Buch, sogar in Deutsch, hier: https://git-scm.com/book/de/v2, übrigens auch als PDF zum Download. 

Eine git Einführung für Entwickler (ohne speziellen Bezug zu Python) gibt es auf der Seite https://realpython.com/python-git-github-intro/. Die haben auch eine Einführung in Continuous Integration mit Python Beispielen: https://realpython.com/python-continuous-integration/.

Die Gemeinsamkeiten und Unterschiede von Github und Gitlab lassen sich auch mit Python-Beispielen erkunden. Ich stelle deshalb einige Module vor, die mit diesen Umgebungen kommunizieren.


<h2>GitPython</h2>

GitPython ist ein Modul, das eine allgemeine git Schnittstelle zur Verfügung stellt, unabhängig von der Umgebung, in der das Repository liegt. Ein ausführliches Tutorial findet man hier: https://gitpython.readthedocs.io/en/stable/tutorial.html, weitere Infos findet man in zahlreichen Artikeln im Internet, z. B. 
* https://azzamsa.com/n/gitpython-intro/
* https://www.devdungeon.com/content/working-git-repositories-python 

<h2>PyGithub</h2>

PyGithub ist die meistverwendete Python API für Github. Dokumentation findet man hier: https://pygithub.readthedocs.io/en/latest/reference.html, zusätzliche Infos in zahlreichen Artikeln im Internet, z. B. 
* https://towardsdatascience.com/all-the-things-you-can-do-with-github-api-and-python-f01790fca131
* https://chase-seibert.github.io/blog/2016/07/22/pygithub-examples.html
* https://www.thepythoncode.com/article/using-github-api-in-python

Bevor wir mit Github kommunizieren können, müssen wir ein Access Token generieren lassen. Das geht auf der Seite https://github.com/settings/tokens. Man wird dann gefragt, welche Typen von Aktionen (scopes) man mit dem Token machen will; man sollte vorsichtig sein bei Aktionen, die mit write:, delete: oder admin: beginnen, denn mit diesen kann man mehr kaputt machen. Information zu den Scopes gibt es hier: https://developer.github.com/apps/building-oauth-apps/understanding-scopes-for-oauth-apps/. 

In [ ]:
# Das Modul müssen wir erst installieren
!pip install PyGithub


In [ ]:
# Und importieren
from github import Github

# Dann lese ich den Access Token aus einer Datei
with open("github-access-token.txt", "r") as fh:
    my_token = fh.readline().strip()       # strip() vermeidet Newline-Probleme

# Mit dem Token kann ich ein Github Objekt erzeugen
g = Github(my_token)

# Zeige die Repositories eines Benutzers
user = g.get_user("michael-wettach")
for repo in user.get_repos():
    print(repo)

# Lies eine Datei aus einem Repository
repo = g.get_repo("michael-wettach/pythonsamples")
file = repo.get_contents("README.md", ref="main")    # hole aus branch "main"
data = file.decoded_content.decode("utf-8")          # und lies den Text
print(data)

Repository(full_name="michael-wettach/pythonsamples")
# pythonsamples
Dieses Repository ist eine Sammlung von Python-Codebeispielen, die ich 
(auf Basis von Internet-Artikeln, Tutorials und anderen Code-Beispielen) 
entwickelt habe zur Weitergabe von Python Know-how in einem Projekt.



<h2>python-gitlab</h2>

python-gitlab ist die Python API für Gitlab. Dokumentation zum Loslegen findet man hier https://python-gitlab.readthedocs.io/en/stable/api-usage.html, zusätzliche Infos in einigen Artikeln im Internet. Ich habe nicht so viele allgemeine Artikel und Beispiele gefunden (die meisten behandeln Continuous Integration), aber z. B. diese:
* Einführungs-Video https://youtu.be/LhVKAwvOg9o
* https://dzone.com/articles/using-gitlab-rest-api-to-create-projects 
* Code-Beispiele https://www.programcreek.com/python/?action=search_nlp&CodeExample=gitlab&submit=Search
* Kommentare in Chinesisch, aber der Code spricht für sich selbst: https://blog.csdn.net/xie_0723/article/details/75215869

<br/>Gitlab Continous Integration: 
* https://machine-learning-blog.de/2019/10/10/continuous-integration-tutorial-gitlab-cicd-einrichten/ 
* https://dev.to/ruanbekker/setup-a-basic-ci-pipeline-on-gitlab-3ndh
* https://www.section.io/engineering-education/building-a-cicd-pipeline-using-gitlab/
* https://dzone.com/articles/creating-a-devops-toolbox-for-gitlab-ci
* https://medium.com/cubemail88/setting-gitlab-ci-cd-for-python-application-b59f1fb70efe

Auch in Gitlab braucht man einen persönlichen access token, den man sich hier erstellen kann: https://gitlab.com/-/profile/personal_access_tokens 

In [ ]:
# Das Modul müssen wir erst installieren
!pip install python-gitlab

In [47]:
# und importieren
import gitlab
import base64

# Dann lese ich den Access Token aus einer Datei
with open("gitlab-access-token.txt", "r") as fh:
    my_token = fh.readline().strip()       # strip() vermeidet Newline-Probleme

# Mit dem Token kann ich ein Gitlab Objekt erzeugen.
gl = gitlab.Gitlab('https://gitlab.com/', private_token=my_token)

# Suche nach einem bestimmten User
# https://python-gitlab.readthedocs.io/en/stable/gl_objects/users.html
user = gl.users.list(username='mwettach')[0]
print(user.id, user.name, user.web_url)           # name könnte ich auch ändern

# Suche nach einem bestimmten Projekt
# https://python-gitlab.readthedocs.io/en/stable/gl_objects/projects.html
for project in gl.projects.list(search='pythonsamples'):
    if project.path_with_namespace == 'mwettach/pythonsamples':
        print(project)
        print(project.id, project.name, project.namespace["name"])
        my_project = project

# Lies eine Datei aus einem Projekt
for item in my_project.repository_tree(path='.', ref='main'):
    if item['name'] == 'README.md':
        file_info = my_project.repository_blob(item['id'])
        content = base64.b64decode(file_info['content'])
        print(content.decode('utf-8'))


9242560 Michael Wettach https://gitlab.com/mwettach
<class 'gitlab.v4.objects.projects.Project'> => {'id': 27863540, 'description': 'Zum Ausprobieren der Gitlab-Funktionalität', 'name': 'pythonsamples', 'name_with_namespace': 'Michael Wettach / pythonsamples', 'path': 'pythonsamples', 'path_with_namespace': 'mwettach/pythonsamples', 'created_at': '2021-07-02T14:09:26.981Z', 'default_branch': 'main', 'tag_list': [], 'topics': [], 'ssh_url_to_repo': 'git@gitlab.com:mwettach/pythonsamples.git', 'http_url_to_repo': 'https://gitlab.com/mwettach/pythonsamples.git', 'web_url': 'https://gitlab.com/mwettach/pythonsamples', 'readme_url': 'https://gitlab.com/mwettach/pythonsamples/-/blob/main/README.md', 'avatar_url': None, 'forks_count': 0, 'star_count': 0, 'last_activity_at': '2021-07-02T14:09:26.981Z', 'namespace': {'id': 12607832, 'name': 'Michael Wettach', 'path': 'mwettach', 'kind': 'user', 'full_path': 'mwettach', 'parent_id': None, 'avatar_url': 'https://secure.gravatar.com/avatar/37555bc